In [1]:
# Import the required libraries and dependencies
import pandas as pd
import datetime as dt
from prophet import Prophet
from clean_data import clean_data


In [2]:
# Create path to CSV file
csv_path = "Metro_Interstate_Traffic_Volume.csv"
# Import the CSV into a pandas DataFrame
df = pd.read_csv(csv_path)
# Run imported DataFrame through clean_data function
clean_df = clean_data(df)

In [3]:
sorted_df = clean_df.sort_values(by='date_time', ascending=True)
sorted_df.head(30)


,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,weather_description,traffic_volume,is_holiday
date_time,,,,,,,,,
2012-10-02 09:00:00,NaN,288.28,0.0,0.0,40,Clouds,scattered clouds,5545,False
2012-10-02 10:00:00,NaN,289.36,0.0,0.0,75,Clouds,broken clouds,4516,False
2012-10-02 11:00:00,NaN,289.58,0.0,0.0,90,Clouds,overcast clouds,4767,False
2012-10-02 12:00:00,NaN,290.13,0.0,0.0,90,Clouds,overcast clouds,5026,False
2012-10-02 13:00:00,NaN,291.14,0.0,0.0,75,Clouds,broken clouds,4918,False
2012-10-02 14:00:00,NaN,291.72,0.0,0.0,1,Clear,sky is clear,5181,False
2012-10-02 15:00:00,NaN,293.17,0.0,0.0,1,Clear,sky is clear,5584,False
2012-10-02 16:00:00,NaN,293.86,0.0,0.0,1,Clear,sky is clear,6015,False
2012-10-02 17:00:00,NaN,294.14,0.0,0.0,20,Clouds,few clouds,5791,False


In [4]:
def identify_gaps(df, output_file):
    # Define the expected range of hourly datetimes
    start_date = pd.to_datetime('2012-10-02 09:00:00')
    end_date = pd.to_datetime('2018-09-30 23:00:00')
    expected_dates = pd.date_range(start=start_date, end=end_date, freq='H')

    # Extract the actual datetimes from the DataFrame
    actual_dates = df.index

    # Identify missing datetimes
    missing_dates = expected_dates[~expected_dates.isin(actual_dates)]

    # Export missing datetimes to a CSV file
    missing_dates.to_frame(name='missing_datetime').to_csv(output_file, index=False)

# Assuming 'df' is your DataFrame
output_file = 'missing_datetimes.csv'
identify_gaps(clean_df, output_file)

C:\Users\Matt\AppData\Local\Temp\ipykernel_23904\2311151761.py:5: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  expected_dates = pd.date_range(start=start_date, end=end_date, freq='H')


In [5]:
def identify_gaps(df, output_missing_file, output_long_gaps_file):
    # Define the expected range of hourly datetimes with frequency 'H'
    start_date = pd.to_datetime('2012-10-02 09:00:00')
    end_date = pd.to_datetime('2018-09-30 23:00:00')
    expected_dates = pd.date_range(start=start_date, end=end_date, freq='H')

    # Extract the actual datetimes from the DataFrame
    actual_dates = df.index

    # Identify missing datetimes
    missing_dates = expected_dates[~expected_dates.isin(actual_dates)]

    # Identify gaps longer than 6 hours
    gap_lengths = (missing_dates - missing_dates.shift(1)).fillna(pd.Timedelta(0))
    long_gaps = missing_dates[gap_lengths >= pd.Timedelta(hours=6)]

    # Export missing datetimes to a CSV file
    missing_dates.to_frame(name='missing_datetime').to_csv(output_missing_file, index=False)

    # Export long gaps to a separate CSV file
    long_gaps.to_frame(name='long_gap_datetime').to_csv(output_long_gaps_file, index=False)

# Assuming 'df' is your DataFrame
output_missing_file = 'missing_datetimes.csv'
output_long_gaps_file = 'long_gaps.csv'
identify_gaps(clean_df, output_missing_file, output_long_gaps_file)

C:\Users\Matt\AppData\Local\Temp\ipykernel_23904\2205768346.py:5: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  expected_dates = pd.date_range(start=start_date, end=end_date, freq='H')


NullFrequencyError: Cannot shift with no freq

In [6]:
len(clean_df)

48193

In [7]:
holiday_rows = df['is_holiday']

KeyError: 'is_holiday'

In [ ]:
print(df['holiday'].unique())